In [1]:
import os, sys
dir = os.path.dirname(os.path.abspath(''))
if not dir in sys.path: 
    sys.path.append(dir)
    
import cf_xarray

import xarray as xr
from xreds.utils import load_dataset

In [2]:
dataset_spec = {
"path": "s3://noaa-nos-cora-pds/V1.1/assimilated/500m_grid/zarr/500m_grid_zeta_1979-2022.zarr",
"type": "zarr",
"chunks": {
    "time": 24,
    "node": 160000
}
}

ds = load_dataset(dataset_spec)
ds

<xarray.Dataset> Size: 3TB
Dimensions:  (time: 385704, nodes: 1101323)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2022-12-31T23:00:16
    lat      (nodes) float32 4MB dask.array<chunksize=(1101323,), meta=np.ndarray>
    lon      (nodes) float32 4MB dask.array<chunksize=(1101323,), meta=np.ndarray>
Dimensions without coordinates: nodes
Data variables:
    node     (nodes) float64 9MB dask.array<chunksize=(1101323,), meta=np.ndarray>
    zeta     (nodes, time) float64 3TB dask.array<chunksize=(10000, 24), meta=np.ndarray>
Attributes: (12/51)
    Conventions:                   ACDD-1.3, CF-1.7
    NCO:                           netCDF Operators version 5.1.5 (Homepage =...
    acknowledgment:                NOAA’s Coastal Ocean Reanalysis wouldn’t b...
    atmospheric_forcing:           ERA5
    comment:                       CORA-GEC v1.1 is the only dataset suitable...
    contact:                       tide.predictions@noaa.gov
    ...                            ...
    time_coverage_duration:        P1D
    time_coverage_end:             1979-01-01
    time_coverage_resolution:      hourly
    time_coverage_start:           1979-01-01
    title:                         CORA-GEC Posterior
    version:                       noaa.stofs.2d.glo.v1.1.0r2-21-g7052115

In [3]:
import xpublish_wms.grids
from xpublish_wms.wms.get_map import WMSGetMapQuery, GetMap

query = WMSGetMapQuery(
    service="WMS",
    version="1.3.0",
    request="GetMap",
    layers="zeta",
    crs="EPSG:3857",
    styles="raster/default",
    width=512,
    height=512,
    bbox="-10018754.171394622,0,-5009377.085697312,5009377.085697312",
    colorscalerange="-0.7599609222535499,5.04364740067773",
    time="2022-12-31T23:00:00Z",
    elevation=None,
    tile=None,
    autoscale=False
)

getmap_service = GetMap(array_render_threshold_bytes=1e9)

# Decode request params
getmap_service.ensure_query_types(ds, query, {})

# Select data according to request
da = getmap_service.select_layer(ds)
da = getmap_service.select_time(da)
da = getmap_service.select_elevation(ds, da)
da = getmap_service.select_custom_dim(da)

da

<xarray.DataArray 'zeta' (nodes: 1101323)> Size: 9MB
dask.array<getitem, shape=(1101323,), dtype=float64, chunksize=(10000,), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] 8B 2022-12-31T23:00:16
    lat      (nodes) float32 4MB dask.array<chunksize=(1101323,), meta=np.ndarray>
    lon      (nodes) float32 4MB dask.array<chunksize=(1101323,), meta=np.ndarray>
Dimensions without coordinates: nodes
Attributes:
    coverage_content_type:  modelResult
    long_name:              water surface elevation above geoid
    standard_name:          sea_surface_height_above_geoid
    units:                  m

In [34]:
import numpy as np

np.array((da.cf["longitude"], da.cf["latitude"],da.values)).T

array([[-8.29560242e+001,  2.46289501e+001,  0.00000000e+000],
       [-8.29511948e+001,  2.46283226e+001,  0.00000000e+000],
       [-8.29463654e+001,  2.46276970e+001,  0.00000000e+000],
       ...,
       [-6.46476059e+001,  1.83643646e+001,  1.60523034e+046],
       [-6.46432953e+001,  1.83628445e+001, -4.74424734e-248],
       [-6.46389847e+001,  1.83613262e+001, -3.26709911e+132]])

In [ ]:
np.meshgrid(da.cf["longitude"], da.cf["latitude"])

MemoryError: Unable to allocate 4.41 TiB for an array with shape (1101323, 1101323) and data type float32

In [35]:
import uuid
uuid.uuid4()

UUID('52a1b88c-f416-4737-8374-73425e912fef')

In [6]:
getmap_service.get_map(ds, query, {})
ds.gridded.name

Error rendering quadmesh: 'QuadMeshRectilinear' object has no attribute 'required_columns', falling back to raster


ValueError: Raster aggregation expects a 2D or 3D DataArray, found 1 dimensions

In [5]:
import numpy as np
import dask.array as dask_array
from xpublish_wms.utils import to_mercator, to_lnglat

crs = getmap_service.crs
bbox = getmap_service.bbox

if crs == "EPSG:3857":
    bbox = to_lnglat.transform([bbox[0], bbox[2]], [bbox[1], bbox[3]])
    bbox = np.array(bbox).flatten().tolist()

adjust_lng = 0
if np.min(da.cf["longitude"]) < -180:
    adjust_lng = 360
elif np.max(da.cf["longitude"]) > 180:
    adjust_lng = -360

x = da.cf["longitude"] + adjust_lng
y = da.cf["latitude"]

x = np.where((x >= bbox[0]) & (x <= bbox[2]))[0]
y = np.where((y >= bbox[1]) & (y <= bbox[3]))[0]

e_ind = np.intersect1d(x, y)

# da = da.isel(node=e_ind)

ds.element[np.any(np.isin(ds.element.values.flat, e_ind).reshape(ds.element.shape), axis=1)]


KeyboardInterrupt



[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
